<div style='text-align: center;'>
<img src="images/math60082-banner.png" alt="image" width="80%" height="auto">
</div>

# Lab Workbook - Week 9


In [ ]:
# To get started import some libraries
import numpy
import matplotlib.pyplot as plt

In [ ]:
# setup the parameters
sigma = 0.4
r = 0.05
X = 2
T = 1
imax = 4
jmax = 4

In [ ]:
## calculate the step size and then setup storage for the value
dS = 2*X / jmax
dt = T / imax

S = numpy.zeros(jmax+1)
t = numpy.zeros(imax+1)
vNew = numpy.zeros(jmax+1)
vOld = numpy.zeros(jmax+1)

# calculate the values of S_j and t^i and check they work as expected
for i in range(imax+1):
    t[i] = i*dt

for j in range(jmax+1):
    S[j] = j*dS

# first enter the value of the option at expiry
for j in range(jmax+1):
    vOld[j] = max( X - S[j] , 0.0 )
    vNew[j] = max( X - S[j] , 0.0 )

# Tasks


1. setup the matrix problem


2. solve the matrix using a sparse solver


In [ ]:
from scipy.linalg import solve_banded

In [ ]:
A_banded = numpy.zeros(shape=(3,jmax+1))
u=1
l=1
l_and_u = (l, u)

# b_0 V_0 + c_0 V_1 = d_0
A_banded[1][0] = 1.0 # b_0
A_banded[0][1] = 0.0 # c_0
# populate middle rows
for j in range(1,jmax):
    # a_j V_j-1 + b_j V_j + c_i V_j+1 = d_i
    A_banded[2][j-1] = 0.25*(sigma*sigma*j*j - r*j) # a_j
    A_banded[1][j] = -1./dt - 0.5*sigma*sigma*j*j - 0.5*r # b_j
    A_banded[0][j+1] = 0.25*(sigma*sigma*j*j + r*j) # c_j
# populate last row
# a_jmax V_jmax-1 + b_jmax V_jmax = d_jmax
A_banded[2][jmax-1] = 0.0 # a_jmax 
A_banded[1][jmax] = 1.0 # b_jmax

# print banded matrix
print(A_banded)

In [ ]:
for i in range(imax-1,-1,-1):
    # Create a vector for multiplication
    d = numpy.zeros(jmax+1)
    d[0] = X*numpy.exp(-r*(T-i*dt))
    for j in range(1,jmax):
        aa = 0.25*(sigma*sigma*j*j - r*j) # a_j
        bb = 1./dt - 0.5*sigma*sigma*j*j - 0.5*r # b_j + 2./dt
        cc = 0.25*(sigma*sigma*j*j + r*j) # c_j
        d[j] = - aa*vOld[j-1] - bb*vOld[j] - cc*vOld[j+1]
    d[jmax] = 0.0

    # Perform sparse matrix-vector multiplication
    vNew = solve_banded(l_and_u,A_banded,d)
    
    vOld = numpy.copy(vNew)

In [ ]:
vNew

## Matrix Form


3. Try different values of `imax` and `jmax` -- what do you notice?


4. Put your algorithm in a function, and return the solution:
    - Parameters: `sigma`, `r`, `X`, `T`, `imax`, `jmax`
    - return: `vNew`, `S`


5. Try plotting the solution $V(S,t=0)$ against $S$ for different values of $imax$ and $jmax$


## Stability and Convergence of the Scheme


6. derive the truncation errors of the scheme


7. by choosing different values of $\Delta t$ and $\Delta S$, verify the convergence rate of the method


8. analyse the errors, comparing with the analytic solution


9. try interpolation techniques to get the values in between the grid points.


10. use `numba` to speed up your code
